In [1]:
import get_data as gd
from utils import get_window_of_day, get_closest_station, get_month_window
import pymortar
import datetime
import pandas as pd
import json

In [2]:
dates = get_window_of_day('2018-07-20')
client = pymortar.Client()

In [4]:
dates

('2018-07-20T00:00:00-07:00', '2018-07-21T00:00:00-07:00')

In [5]:
start = "2016-01-01T00:00:00Z"
end = "2018-12-21T00:00:00Z"
site = "ciee"
data = gd.get_df(site, start, end, client)

In [6]:
data.head()

,power,weather
2015-12-31 16:00:00-08:00,3248.0,48.2
2015-12-31 16:15:00-08:00,2592.0,NaN
2015-12-31 16:30:00-08:00,2416.0,NaN
2015-12-31 16:45:00-08:00,2288.0,NaN
2015-12-31 17:00:00-08:00,2272.0,NaN


In [7]:
import static_models as sm

DEBUG:matplotlib:CACHEDIR=/Users/jleyden/.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /Users/jleyden/.matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [8]:
pdp_days =  [
        "2018-06-12",
        "2018-06-13",
        "2018-07-10",
        "2018-07-16",
        "2018-07-17",
        "2018-07-19",
        "2018-07-24",
        "2018-07-25",
        "2018-07-27"
    ]

In [13]:
import datetime
pdp_days = pd.to_datetime(pdp_days).date

In [14]:
sm.power_model(pdp_days[-1], data, pdp_days, )

(array([[2080., 1984., 1984., 2096., 2048., 1968., 2032., 2016., 2032.,
         1968., 2000., 1952., 1952., 1936., 1920., 1936., 1984., 1936.,
         1904., 1952., 1968., 1936., 2016., 1952., 2000., 1968., 2016.,
         1904., 1888., 2048., 2096., 2432., 2608., 2864., 3008., 3040.,
         3232., 3648., 3600., 3600., 3760., 4048., 4096., 4128., 4256.,
         4288., 4240., 4192., 4400., 4544., 4016., 4496., 4176., 4496.,
         4096., 4640., 4288., 4656., 4144., 4896., 4288., 5296., 4224.,
         4144., 4960., 4672., 5152., 5104., 3968., 4384., 4704., 4176.,
         3536., 4080., 3152., 3312., 2848., 3088., 3104., 3040., 2736.,
         3136., 3440., 3472., 2944., 2704., 2576., 2768., 2800., 2704.,
         2720., 2528., 2944., 2816., 2592., 2416.]]),
 array([1760., 1920., 1760., 1760., 1760., 1920., 1920., 1920., 1920.,
        1760., 2080., 2400., 2240., 2400., 2080., 1920., 1760., 1760.,
        1760., 1920., 1760., 1760., 2400., 2400., 2400., 2400., 2400.,
        2400.

In [5]:
from get_test_days import get_test_data

DEBUG:matplotlib:CACHEDIR=/Users/jleyden/.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /Users/jleyden/.matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [7]:
get_test_data("avenal-animal-shelter", pdp_days, "2016-01-01T00:00:00-08:00", "2019-03-01T00:00:00-08:00", client)

([datetime.date(2016, 7, 15),
  datetime.date(2016, 6, 29),
  datetime.date(2016, 7, 26),
  datetime.date(2016, 7, 27),
  datetime.date(2017, 7, 28),
  datetime.date(2016, 7, 25)],
 [datetime.date(2017, 7, 17),
  datetime.date(2016, 6, 28),
  datetime.date(2018, 8, 1),
  datetime.date(2016, 7, 28),
  datetime.date(2016, 7, 29)])

 **Existing**
 
 Piece-wise temperature: Temperature split into intervals. Given a temperature in degrees Fahrenheit $t$, let $f_j = \min(t - C_j, C_{j+1} - C_{j})$, where $C = [0, 40, 50, 60, 70, 80, \infty], j \in [0, 5]$ (six features total).

**Correct**

Piece-wise temperature: Temperature split into intervals. Given a temperature in degrees Fahrenheit $t$, let $f_j = \max(0, \min(t - C_j, C_{j+1} - C_{j}))$, where $C = [0, 40, 50, 60, 70, 80, \infty], j \in [0, 5]$ (six features total).

In [3]:
PDP=[datetime.date(2017, 6, 16),
 datetime.date(2017, 6, 19),
 datetime.date(2017, 6, 20),
 datetime.date(2017, 6, 22),
 datetime.date(2017, 6, 23),
 datetime.date(2017, 7, 7),
 datetime.date(2017, 7, 27),
 datetime.date(2017, 7, 31),
 datetime.date(2017, 8, 1),
 datetime.date(2017, 8, 2)]

In [9]:
import static_models as sm
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import pymortar
import logging 
import os
import json

from get_test_days import get_test_data
from feature_engineering import get_time_of_week, get_t_cutoff_values
from utils import get_window_of_day, get_workdays, get_closest_station, mean_absolute_percentage_error
from daily_data import get_daily_data
import get_data as gd

In [10]:
start = "2016-01-01T00:00:00Z"
end = "2018-12-21T00:00:00Z"
site = "ciee"
cli = pymortar.Client()

interval = '15min'
agg = 'RAW'
alphas = [0.0001, .001, 0.01, 0.05, 0.1, 0.5, 1, 10]

# Get weather
weather = gd.get_weather(site, start, end, agg=agg, window=interval, cli=cli)
weather.index = weather.index.tz_localize('UTC').tz_convert('US/Pacific')
closest_station = get_closest_station(site)
if closest_station is not None:
    weather = pd.DataFrame(weather[closest_station])
else:
    weather = pd.DataFrame(weather.mean(axis=1))

# Get power
power = gd.get_power(site, start, end, agg=agg, window=interval, cli=cli) * 4
power.index = power.index.tz_localize('UTC').tz_convert('US/Pacific')

# Merge
weather_mean = pd.DataFrame(weather.mean(axis=1))
power_sum = pd.DataFrame(power.sum(axis=1))
data = power_sum.merge(weather_mean, left_index=True, right_index=True)
data.columns = ['power', 'weather']

/Users/jleyden/anaconda3/envs/mortar/lib/python3.6/site-packages/pandas/core/nanops.py:358: RuntimeWarning: divide by zero encountered in true_divide
  the_mean = the_sum / count


In [13]:
sm.power_model(datetime.date(2017, 10, 26), data, PDP)

(array([[ 4368.,  4320.,  4032.,  4224.,  3288.,  3504.,  3384.,  3864.,
          3144.,  3336.,  3240.,  3024.,  3072.,  2736.,  3120.,  2952.,
          3000.,  2928.,  2808.,  2952.,  2880.,  2808.,  3048.,  2784.,
          2904.,  3264.,  3000.,  2856.,  2832.,  2832.,  2880.,  3312.,
          3744.,  3912.,  4704.,  4680.,  5496.,  5856.,  6984.,  7488.,
          9216.,  9480.,  9384., 11256., 10752., 11064., 11688., 12000.,
         13464., 11904., 11376., 13224., 13872., 14184., 14712., 13920.,
         14928., 15552., 15360., 14616., 15480., 15504., 14400., 15960.,
         14616., 15264., 12240., 13056., 11400., 11184., 11880., 10896.,
          9048.,  8160.,  9504.,  9048.,  8328.,  7680.,  7392.,  6360.,
          6912.,  7392.,  7704.,  6912.,  7848.,  6024.,  6696.,  6096.,
          6408.,  5208.,  5808.,  5208.,  5232.,  4608.,  5136.,  5352.]]),
 array([ 3520.,  4800.,  2720.,  4480.,  1760.,  3680.,  1600.,  2880.,
         2400.,  1440.,  2720.,  2240.,  1600.,  

In [14]:
start = start_train
end = end_train
interval = '15min'
agg = 'MEAN'
alphas = [0.0001, .001, 0.01, 0.05, 0.1, 0.5, 1, 10]

# Get weather
weather = gd.get_weather(site, start, end, agg=agg, window=interval, cli=cli)
weather.index = weather.index.tz_localize('UTC').tz_convert('US/Pacific')
closest_station = get_closest_station(site)
if closest_station is not None:
    weather = pd.DataFrame(weather[closest_station])
else:
    weather = pd.DataFrame(weather.mean(axis=1))

# Get power
power = gd.get_power(site, start, end, agg=agg, window=interval, cli=cli) * 4
power.index = power.index.tz_localize('UTC').tz_convert('US/Pacific')

# Merge
weather_mean = pd.DataFrame(weather.mean(axis=1))
power_sum = pd.DataFrame(power.sum(axis=1))
data = power_sum.merge(weather_mean, left_index=True, right_index=True)
data.columns = ['power', 'weather']

INFO:root:Generating a new JWT token. Your old token may have expired
/Users/jleyden/anaconda3/envs/mortar/lib/python3.6/site-packages/pandas/core/nanops.py:358: RuntimeWarning: divide by zero encountered in true_divide
  the_mean = the_sum / count


In [15]:
data

,power,weather
2015-12-31 16:00:00-08:00,3248.0,48.20
2015-12-31 16:15:00-08:00,2592.0,NaN
2015-12-31 16:30:00-08:00,2416.0,NaN
2015-12-31 16:45:00-08:00,2288.0,NaN
2015-12-31 17:00:00-08:00,2272.0,NaN
2015-12-31 17:15:00-08:00,2576.0,NaN
2015-12-31 17:30:00-08:00,2368.0,NaN
2015-12-31 17:45:00-08:00,2592.0,46.40
2015-12-31 18:00:00-08:00,1616.0,NaN
2015-12-31 18:15:00-08:00,1936.0,NaN


In [17]:
from datetime import timedelta

In [18]:
date = pd.to_datetime('2018-07-20').date()
date - timedelta(days=30)

datetime.date(2018, 6, 20)

In [2]:
get_month_window('2018-07-20')

('2018-06-20T00:00:00-07:00', '2018-07-20T00:00:00-07:00')

In [1]:
from model_objects import WeatherModel

In [2]:
wm = WeatherModel(10, 10, [], 5)

In [3]:
wm

In [4]:
{wm: 5}

{<model_objects.WeatherModel at 0x1a217862b0>: 5}

In [5]:
wm_2 = WeatherModel(10, 5, [], 5)

In [8]:
dic = {
    wm: 5,
    wm_2: 2
}

In [12]:
f = open('./config.json')

In [19]:
import pickle

In [26]:
f = open('new.txt', 'wb')

In [27]:
pickle.dump(wm, f)

In [31]:
f = open('new.txt', 'rb')

In [32]:
load = pickle.load(f)

In [33]:
load

In [34]:
import json

In [49]:
obj = {'hi': 2, 'bye': 3}

In [50]:
f = open('new.json', 'w')

In [52]:
json.dump(obj, f)

In [53]:
f = open('./models/avenal-animal-shelter/best.txt', 'rb')

In [54]:
load = pickle.load(f)

In [56]:
load.rmse

1576.3491231421367

In [1]:
from evaluate import evaluate

In [2]:
evaluate('ciee', '2018-07-20')

{'site': 'ciee',
 'date': datetime.date(2018, 7, 20),
 'cost': {'actual': 20.015674666666666, 'baseline': 17.91577},
 'degree-days': {'cooling': None, 'heating': None},
 'baseline-type': 'Power Model: 3 out of 10 last days',
 'baseline-rmse': 1634.904888259261,
 'actual': array([1866.66666667, 1920.        , 1866.66666667, 1866.66666667,
        1866.66666667, 1813.33333333, 1973.33333333, 1813.33333333,
        1866.66666667, 1760.        , 1866.66666667, 1760.        ,
        1813.33333333, 1866.66666667, 1973.33333333, 2080.        ,
        2026.66666667, 1973.33333333, 1866.66666667, 1760.        ,
        1866.66666667, 1706.66666667, 1866.66666667, 1813.33333333,
        2026.66666667, 2026.66666667, 2080.        , 1973.33333333,
        2186.66666667, 2186.66666667, 2400.        , 2986.66666667,
        2826.66666667, 2933.33333333, 2880.        , 3200.        ,
        3626.66666667, 3786.66666667, 3573.33333333, 3840.        ,
        4213.33333333, 4480.        , 4533.33333

In [1]:
from model_objects import PowerModel

In [2]:
site = 'ciee'
X = 10
Y = 10
PDP_dates =  [
        "2018-06-12",
        "2018-06-13",
        "2018-07-10",
        "2018-07-16",
        "2018-07-17",
        "2018-07-19",
        "2018-07-24",
        "2018-07-25",
        "2018-07-27"
    ]
pm = PowerModel(X, Y, PDP_dates)

In [3]:
import pymortar
import pandas as pd

In [1]:
from evaluate import evaluate
import matplotlib.pyplot as plt

DEBUG:matplotlib:CACHEDIR=/Users/jleyden/.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /Users/jleyden/.matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
day = '2018-07-24'
site = 'orinda-public-library'
result = evaluate(site, day)
result

TypeError: to_indexed_series() missing 1 required positional argument: 'date'

In [29]:
import pandas as pd

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from pandas.tseries.offsets import CustomBusinessDay
import matplotlib.pyplot as plt

import numpy as np
import datetime

import get_data as gd

import pymortar

def _remove_PDP_days(data, PDP_list):
    
    try:
        for i in PDP_list:
            day=datetime.datetime.strptime(str(i), "%Y-%m-%d").date()
            data=data[~(data.index.date == day)]       
        return data
    
    except:
        print('error in _remove_PDP_days')
        return data
    

def _remove_WE_holidays_NaN(data, start, end):
    
    no_WE = ~((data.index.weekday == 5) | (data.index.weekday == 6)) # remove if WE

    cal = calendar() 
#     start = datetime.datetime.strftime(data.index.min(),"%Y-%m-%d")
#     end =datetime.datetime.strftime(data.index.max(),"%Y-%m-%d")

    hol_cal = cal.holidays(start=start, end=end)
    #hol_cal=pd.to_datetime(hol_cal).tz_localize("America/Los_Angeles") 
    no_hol = ~data.index.isin(hol_cal) # remove if it is a national holiday   
    no_NaN = ~data.isna().all(axis=1) # remove if has any NaN for any hour

    
    return data[no_WE & no_hol & no_NaN]

def get_window_of_day(date):
    start, end = pd.date_range(start=date, periods=2, freq='1d')
    start_ts = str(start.date()) + 'T00:00:00-08:00'
    end_ts = str(end.date()) + 'T00:00:00-08:00'
    return start_ts, end_ts

def isValidTestDay(date):
    start, end = get_window_of_day(date)
    data  = gd.get_df(site, start, end, agg='MEAN', interval='15min', cli=cli)
    for column in data.columns:
        col = data[column]
        if col.isna().sum() > 0.5*len(result):
            return False
        if (col == 0).sum() > 0.5*len(result):
            return False
        if len(col.unique()) < 3:
            return False
    return True

#%%
def get_test_data(site, PDP_days, start_search, end_search, cli, fraction_test=0.5):
    means=[]
    maxes = []
    for day in PDP_days:
        start, end = get_window_of_day(day)
        weather_mean = gd.get_weather(site, start, end, agg='MEAN', window='24h', cli=cli)
        means.append(weather_mean)   
        weather_max = gd.get_weather(site, start, end, agg='MAX', window='24h', cli=cli)
        maxes.append(weather_max)
        
    means = pd.concat(means, sort=True)
    maxes = pd.concat(maxes, sort=True)
    mean_cutoff = means.median().mean()
    max_cutoff = maxes.median().mean()

    weather_mean_all = gd.get_weather(site, start_search, end_search, agg='MEAN', window='24h', cli=cli)
    weather_mean = weather_mean_all.mean(axis=1)  
    weather_max_all = gd.get_weather(site, start_search, end_search, agg='MAX', window='24h', cli=cli)
    weather_max = weather_max_all.mean(axis=1)
    
    weather = pd.DataFrame({'mean': weather_mean,'max': weather_max})
    
    
    weather=_remove_PDP_days(weather, PDP_days) 
    
    weather=_remove_WE_holidays_NaN(weather, start_search, end_search)
    
    above_mean_cuttoff = weather['mean'] >= mean_cutoff
    above_max_cutoff = weather['max'] >= max_cutoff
    
    above_cutoff = above_max_cutoff & above_mean_cuttoff
    qualified = above_cutoff[above_cutoff]
    valid_filter = [isValidTestDay(day) for day in qualified.index]
    qualified = qualified[valid_filter]
    num_testing_samples=int(np.ceil(np.size(qualified)*fraction_test))
        
    shuffled=list(qualified.sample(frac=1, random_state=47).index)
    weather_test=shuffled[0:num_testing_samples]
    weather_train=shuffled[num_testing_samples:]

    # convert to datetime.dates
    test_days = [t.date() for t in weather_test]
    train_days = [t.date() for t in weather_train]
    
    return test_days, train_days

    

In [30]:
site = 'avenal-veterans-hall'
PDP_dates =  [
        "2018-06-12",
        "2018-06-13",
        "2018-07-10",
        "2018-07-16",
        "2018-07-17",
        "2018-07-19",
        "2018-07-24",
        "2018-07-25",
        "2018-07-27"
    ]
start = "2018-06-01T00:00:00Z"
end = "2018-08-01T00:00:00Z"
cli = pymortar.Client()
get_test_data(site, PDP_dates, start, end, cli, fraction_test=0.5)

([datetime.date(2018, 6, 4), datetime.date(2018, 7, 12)],
 [datetime.date(2018, 7, 13)])

In [3]:
site = 'avenal-veterans-hall'
PDP_dates =  [
        "2018-06-12",
        "2018-06-13",
        "2018-07-10",
        "2018-07-16",
        "2018-07-17",
        "2018-07-19",
        "2018-07-24",
        "2018-07-25",
        "2018-07-27"
    ]
start = "2018-07-20T00:00:00Z"
end = "2018-07-22T00:00:00Z"
cli = pymortar.Client()

In [4]:
result  = gd.get_weather(site, start, end, agg='MEAN', window='15min', cli=cli)
result.index = result.index.tz_localize('UTC').tz_convert('US/Pacific')

In [5]:
data = result.iloc[:, 0]

In [6]:
data.loc[pd.to_datetime('2018-07-20')]

98.9999960000001

In [7]:
data = data[data.index.date == pd.to_datetime('2018-07-20').date()]

In [8]:
len(data.unique()) > 9

False

In [9]:
start, end = get_window_of_day('2018-07-20')

In [10]:
start

'2018-07-20T00:00:00-07:00'

In [11]:
import dateutil.parser as parser


def get_window_of_day_2(date):
    start, end = pd.date_range(start=date, periods=2, freq='1d', tz='US/Pacific')
    start_ts = start.isoformat()
    end_ts = end.isoformat()
    return start_ts, end_ts

In [15]:
result  = gd.get_df(site, start, end, agg='MEAN', interval='15min', cli=cli)

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Received RST_STREAM with error code 2"
	debug_error_string = "{"created":"@1556907654.185065000","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1039,"grpc_message":"Received RST_STREAM with error code 2","grpc_status":13}"
>

In [14]:
start, end = get_window_of_day('2018-07-20')
result  = gd.get_df(site, start, end, agg='MEAN', interval='15min', cli=cli)
weather = result['weather']
len(weather.unique()) < 0.2*len(result)

KeyboardInterrupt: 

In [19]:
def isValidTestDay(date):
    start, end = get_window_of_day(date)
    data  = gd.get_df(site, start, end, agg='MEAN', interval='15min', cli=cli)
    for column in data.columns:
        col = data[column]
        if col.isna().sum() > 0.5*len(result):
            return False
        if (col == 0).sum() > 0.5*len(result):
            return False
        if len(col.unique()) < 3:
            return False
    return True

In [23]:
isValidTestDay('2018-07-20')

False

In [16]:
for column in result.columns:
    col = result[column]
    if col.isna().sum() > 0.5*len(result):
        return False
    if (col == 0).sum() > 0.5*len(result):
        return False
    if len(col.unique()) < 3:
        return False

SyntaxError: 'return' outside function (<ipython-input-16-df5ef60ba046>, line 4)

In [4]:
date = pd.to_datetime('2018-07-12').tz_localize('US/Pacific')
date

Timestamp('2018-07-12 00:00:00-0700', tz='US/Pacific')

In [93]:
pd.to_datetime(start).tz_localize('US/Pacific')

Timestamp('2018-07-12 00:00:00-0700', tz='US/Pacific')

In [86]:
start = '2018-7-12'

In [87]:
date = pd.to_datetime(start)

In [89]:
date = pd.date_range(start=date, periods=2, freq='1d', tz='US/Alaska')[0]
date

Timestamp('2018-07-12 00:00:00-0800', tz='US/Alaska', freq='D')

In [80]:
offset = date.dst()

In [83]:
offset.seconds == 3600

True